# Maximização de Influência em Redes Sociais
## João Victor Galindo Borges do Rego
### 1621146

In [ ]:
# import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Building a graph for testing
### Choose a set test graph of 15 nodes or generate your own randomly

In [ ]:
# build a random upper triangular matrix dataframe

a = np.empty(15)
b = np.arange(start=0, stop=15, step=1)
ind = np.arange(len(a))
np.put(a, ind, b)

# randomly generated matrix 
#   uncomment if you want to generate your own 15 x 15 random matrix
# matrixDataframe = pd.DataFrame(np.triu(np.random.choice(4, size=(15, 15), p=[0.8, 0.1, 0.05, 0.05])), columns=a, index=a)

# set 15 x 15 matrix
#   this was originally generated randomly
#   used for thesis
matrixDataframe = pd.DataFrame(np.array([[0,0,2,0,0,2,0,0,0,1,0,0,0,0,0],
                                         [0,0,1,0,0,0,0,0,0,2,0,0,0,0,0],
                                         [0,0,0,1,0,0,0,0,0,2,0,1,0,3,0],
                                         [0,0,0,0,1,0,0,0,0,0,0,1,0,0,0],
                                         [0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
                                         [0,0,0,0,0,0,3,0,2,0,0,1,0,0,0],
                                         [0,0,0,0,0,0,2,0,0,0,2,0,1,0,1],
                                         [0,0,0,0,0,0,0,0,2,0,0,2,0,0,0],
                                         [0,0,0,0,0,0,0,0,0,1,0,0,1,0,0],
                                         [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
                                         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
                                         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]))

matrixDataframe

In [ ]:
# convert dataframe to graph

G = nx.from_numpy_matrix(matrixDataframe.values)
G.remove_edges_from(nx.selfloop_edges(G))

print(G.number_of_nodes(), G.number_of_edges())

In [ ]:
# draw graph

# subax1 = plt.subplot()
# nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')

elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] > 2]
emedium = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] > 1 and d["weight"] <= 2]
esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 1]

pos = nx.kamada_kawai_layout(G)  # positions for all nodes - seed for reproducibility

# nodes
nx.draw_networkx_nodes(G, pos)

# edges
nx.draw_networkx_edges(G, pos, edgelist=elarge, width= 2, alpha=1, edge_color="r")
nx.draw_networkx_edges(G, pos, edgelist=emedium, width= 2, alpha=1, edge_color="g")
nx.draw_networkx_edges(G, pos, edgelist=esmall, width= 2, alpha=0.75, edge_color="b")

# node labels
nx.draw_networkx_labels(G, pos, font_size=12, font_family="sans-serif")
# edge weight labels
edge_labels = nx.get_edge_attributes(G, "weight")
nx.draw_networkx_edge_labels(G, pos, edge_labels)

In [ ]:
# export graph for better visualization
#   only necessary for really big graphs

nx.write_graphml(G, './Graph.graphml')

# Community Based Influence Maximization
### Set your own values for *k* and *δ* below and run all remaining cells
### Relevant data printed at the very bottom of this document

In [ ]:
# CBIM parameters

# k
seedNodeAmount = 2

# δ
mergingIndexThreshold = 0.4

print("k = " + str(seedNodeAmount))
print("δ = " + str(mergingIndexThreshold))

In [ ]:
# community based influence maximization algorithm
# Phase 1 - Initial communities detection

import copy
import random

# 1. Initialize the variables 'NS' and 'CSinit'

NS = list(G.nodes)
CSinit = []


while len(NS) > 0:

    # 2. Select the highest degree nodes 'Dv'

    degreeList = G.degree(weight="weight")
    highestDegree = 0
    highestDegreeNodes = []
    for node in degreeList:
        if node[1] == highestDegree and NS.count(node[0]) > 0:
            highestDegreeNodes.append(node)
        elif node[1] > highestDegree and NS.count(node[0]) > 0:
            highestDegree = node[1]
            highestDegreeNodes = []
            highestDegreeNodes.append(node)


    # 3. Randomly select a node 'v' from 'Dv'

    print("The highest degree nodes are: " + str(highestDegreeNodes))
    v, _ = random.choice(highestDegreeNodes)
    print("The chosen highest degree (v) node is: " + str(v))


    # 4. Get the most similar neighbours of 'v' using Dice neighbour similarity

    highestDSC = .0
    highestDSCNodes = []
    for u in G.neighbors(v):
        DSC = (2 * len(list(nx.common_neighbors(G, u, v)))) / (len(list(G.neighbors(u))) + len(list(G.neighbors(v))))
        if DSC == highestDSC:
            highestDSCNodes.append(u)
        elif DSC > highestDSC:
            highestDSC = DSC
            highestDSCNodes = []
            highestDSCNodes.append(u)


    # 5. Randomly select a neighbour 'sn' from the most similar neighbours list

    print("The most similar neighbour nodes are: " + str(highestDSCNodes))
    sn = random.choice(highestDSCNodes)
    print("The chosen neighbour node (sn) is: " + str(sn))


    # 6. IF 'sn' is not in any community THEN

    nodeIsPresent = False
    for C in CSinit:

        # 10. ELSE
        # 11. Find the community to which 'sn' belongs to and denote it as 'C'
        print(C)
        print(C.count(sn))
        if C.count(sn) > 0:

            # 12. Insert node 'v' into 'C' and remove node 'v' from 'NS'
            C.append(v)
            NS.remove(v)

            nodeIsPresent = True

    if not nodeIsPresent:
        # 7. Create a new community and assign 'v' and 'sn' to it
        community = [v, sn]

        # 8. Insert the new community into community structure
        CSinit.append(community)

        # 9. Remove 'v' and 'sn' from NS
        NS.remove(v)
        NS.remove(sn)


    # 13. Repeat the steps from 2 to 12, until 'NS' = null

In [ ]:
print("Initial communities list (CSinit): " + str(CSinit))

In [ ]:
# Phase 2 - Community consolidation
# 14. Initialize Final Communities (FC)

FC = CSinit.copy()


# 15. Calculate community conductance (γi) and community scale (θi) for each community 'C' in CSinit

EoutList = np.zeros(len(CSinit))
EinList = np.zeros(len(CSinit))
conductanceList = np.zeros(len(CSinit))
scaleList = np.zeros(len(CSinit))
i = 0
for C in CSinit:
    visitedNodes = []
    for node in C:
        visitedNodes.append(node)
        for neighbor in list(G.neighbors(node)):
            if visitedNodes.count(neighbor) == 0:
                if C.count(neighbor) == 0:
                    EoutList[i] += G[node][neighbor]["weight"]
                else:
                    EinList[i] += G[node][neighbor]["weight"]
    conductanceList[i] = EoutList[i] / (2 * EinList[i] + EoutList[i])
    scaleList[i] = len(C) / len(list(G.nodes))
    i += 1

print("Community conductances: " + str(conductanceList))
print("Community scales: " + str(scaleList))


# 16. Calculate the merging index (ψi) for each community in CSinit.

mergingIndexList = np.zeros(len(CSinit))
i = 0
for index in mergingIndexList:
    mergingIndexList[i] = conductanceList[i] * scaleList[i]
    i += 1

print("Community merging indexes: " + str(mergingIndexList))


# 17. Select the community with lowest merging index (Cx)

lowestMergingIndex = (0, 0)
mergingIndexList.sort
i = 0
for index in mergingIndexList:
    if mergingIndexList[i] < lowestMergingIndex[1] or lowestMergingIndex == (0, 0):
        lowestMergingIndex= (i, mergingIndexList[i])
    i += 1


while lowestMergingIndex[1] <= mergingIndexThreshold:
    # 17. Select the community with lowest merging index (Cx)

    lowestMergingIndex = (0, 0)
    mergingIndexList.sort
    i = 0
    for index in mergingIndexList:
        if mergingIndexList[i] < lowestMergingIndex[1] or lowestMergingIndex == (0, 0):
            lowestMergingIndex= (i, mergingIndexList[i])
        i += 1

    print("Lowest merging index community: " + str(lowestMergingIndex[0]))


    # 18. Find the most similar community (Cy) to (Cx) and merge the two communites to form a new community (Cn)

    similarityList = np.zeros(len(CSinit))
    i = 0
    for C in FC:
        DSCsum = 0
        if i != lowestMergingIndex[0]:
            for u in CSinit[lowestMergingIndex[0]]:
                for v in C:
                    DSCsum += (2 * len(list(nx.common_neighbors(G, u, v)))) / (len(list(G.neighbors(u))) + len(list(G.neighbors(v))))
            similarityList[i] = DSCsum / len(CSinit[lowestMergingIndex[0]])
        i += 1

    print("Communities similarity to community " + str(lowestMergingIndex[0]) + ": " + str(similarityList))

    highestSimilarity = (0, 0)
    i = 0
    for similarity in similarityList:
        if similarity > highestSimilarity[1]:
            highestSimilarity = (i, similarity)
        i += 1

    print("Most similar community: " + str(highestSimilarity[0]))

    newCommunity = FC[lowestMergingIndex[0]] + FC[highestSimilarity[0]]

    print("New community: " + str(newCommunity))


    # 19. Calculate the merging index (ψn) for new community (Cn)

    Eout = 0
    Ein = 0
    conductance = 0
    scale = len(newCommunity) / len(list(G.nodes))

    visitedNodes = []
    for node in newCommunity:
            visitedNodes.append(node)
            for neighbor in list(G.neighbors(node)):
                if visitedNodes.count(neighbor) == 0:
                    if C.count(neighbor) == 0:
                        Eout += G[node][neighbor]["weight"]
                    else:
                        Ein += G[node][neighbor]["weight"]

    conductance = Eout / (2 * Ein + Eout)

    print("New community conductance: " + str(conductance))
    print("New community scale: " + str(scale))

    mergingIndex = conductance * scale

    print("New community merging index: " + str(mergingIndex))


    # 20. Replace two communites 'Cx' and 'Cy' with new community 'Cn' in final community set (FC)

    FC[lowestMergingIndex[0]] = newCommunity
    del FC[highestSimilarity[0]]

    print("Final Communities list (FC) so far: " + str(FC))


    # 21. Repeat the process from 17 to 20 until 'ψi' > 'δ'

    lowestMergingIndex = (0, mergingIndex)


print("Final Communities list (FC): " + str(FC))

In [ ]:
# Finding Katz centrality coeficient and seed node selection
# 1. Initialize the Seed node set (SN) and Community set (CS)

SN = np.zeros(len(FC))
CS = FC.copy()


# 2. Calculate Katz centrality for each node

katzCentralityDict = nx.katz_centrality(G, alpha=0.1, beta=0.01, max_iter=100, weight="weight")
katzCentralityList = [(k, v) for k, v in katzCentralityDict.items()]


# 3. Sort all the nodes in each community based on Katz Centrality Coeficient in descending order

katzCentralityList.sort(key=lambda a: a[1], reverse=True)
print("Katz Centrality coeficients for every node: " + str(katzCentralityList))


# 4. Calculate required seed nodes from each community in quota based approach

quotaList = np.zeros(len(CS))
i = 0
for C in CS:
    quotaList[i] = round(seedNodeAmount * (len(C) / len(list(G.nodes))))
    # if quotaList[i] == 0:
    #     quotaList[i] = 1
    i += 1

print("Quotas for every community: " + str(quotaList))
quotaListBackup = quotaList.copy()

# 5. Select the quota number of highest Katz centrality coeficient nodes as seed nodes from each community (Ci)

SN = np.zeros(int(sum(quotaList)))
i = 0
j = 0
for C in CS:
    for node in katzCentralityList:
        if C.count(node[0]) > 0 and quotaList[i] > 0:
            quotaList[i] -= 1
            SN[j] = node[0]
            j += 1
    i += 1

print("Community Set (CS): " + str(CS))
print("Seed Nodes: " + str(SN))

In [ ]:
print("------------------------------")
print("Parameters:") 
print("------------------------------")
print("k = " + str(seedNodeAmount)) 
print("δ = " + str(mergingIndexThreshold))

print("\n")

print("------------------------------")
print("Relevant data for this run:")
print("------------------------------")
print("Initial communities list (CSinit): " + str(CSinit))
print("Final Communities list (FC): " + str(FC))
print("Quotas for every community: " + str(quotaListBackup))
print("Katz Centrality coeficients for every node: " + str(katzCentralityList))
print("Seed Nodes: " + str(SN))